In [55]:
import sqlite3
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
import numpy as np

In [12]:
# Create your connection.
cnx = sqlite3.connect('database.sqlite')
df = pd.read_sql_query("SELECT * FROM Player_Attributes", cnx)

In [13]:
df.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [16]:
df.columns

Index(['overall_rating', 'potential', 'preferred_foot', 'crossing',
       'finishing', 'heading_accuracy', 'short_passing', 'volleys',
       'dribbling', 'curve', 'free_kick_accuracy', 'long_passing',
       'ball_control', 'acceleration', 'sprint_speed', 'agility', 'reactions',
       'balance', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots',
       'aggression', 'interceptions', 'positioning', 'vision', 'penalties',
       'marking', 'standing_tackle', 'sliding_tackle', 'gk_diving',
       'gk_handling', 'gk_kicking', 'gk_positioning', 'gk_reflexes'],
      dtype='object')

In [15]:
# Drop columns which is not related to any relevant feature 
df = df.drop(labels=['id', 'player_fifa_api_id', 'player_api_id', 'date',
                     'attacking_work_rate','defensive_work_rate'],axis=1)

In [17]:
df.fillna(0, inplace=True)

In [21]:
# Convert categorical columns into numerical
pref_foot_dummies = pd.get_dummies(df.preferred_foot, prefix='pref_foot').iloc[:, 1:]


In [22]:
df = pd.concat([df,pref_foot_dummies], axis=1)
df.head()

,overall_rating,potential,preferred_foot,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,...,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,pref_foot_left,pref_foot_right
0,67.0,71.0,right,49.0,44.0,71.0,61.0,44.0,51.0,45.0,...,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0,0,1
1,67.0,71.0,right,49.0,44.0,71.0,61.0,44.0,51.0,45.0,...,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0,0,1
2,62.0,66.0,right,49.0,44.0,71.0,61.0,44.0,51.0,45.0,...,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0,0,1
3,61.0,65.0,right,48.0,43.0,70.0,60.0,43.0,50.0,44.0,...,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0,0,1
4,61.0,65.0,right,48.0,43.0,70.0,60.0,43.0,50.0,44.0,...,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0,0,1


In [23]:
df.columns

Index(['overall_rating', 'potential', 'preferred_foot', 'crossing',
       'finishing', 'heading_accuracy', 'short_passing', 'volleys',
       'dribbling', 'curve', 'free_kick_accuracy', 'long_passing',
       'ball_control', 'acceleration', 'sprint_speed', 'agility', 'reactions',
       'balance', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots',
       'aggression', 'interceptions', 'positioning', 'vision', 'penalties',
       'marking', 'standing_tackle', 'sliding_tackle', 'gk_diving',
       'gk_handling', 'gk_kicking', 'gk_positioning', 'gk_reflexes',
       'pref_foot_left', 'pref_foot_right'],
      dtype='object')

In [46]:
import statsmodels.formula.api as smf
lm = smf.ols(formula = 'overall_rating ~ crossing + potential  + finishing + heading_accuracy + short_passing + \
             volleys + dribbling + curve + free_kick_accuracy + long_passing + ball_control + \
             acceleration + sprint_speed + agility + reactions + balance + shot_power + jumping + \
             stamina + strength + long_shots + aggression + interceptions + positioning + vision + \
             penalties + marking + standing_tackle + sliding_tackle + gk_diving + gk_handling + \
             gk_kicking + gk_positioning + gk_reflexes', data = df).fit()
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         overall_rating   R-squared:                       0.889
Model:                            OLS   Adj. R-squared:                  0.889
Method:                 Least Squares   F-statistic:                 4.321e+04
Date:                Thu, 25 Oct 2018   Prob (F-statistic):               0.00
Time:                        23:45:49   Log-Likelihood:            -4.5073e+05
No. Observations:              183978   AIC:                         9.015e+05
Df Residuals:                  183943   BIC:                         9.019e+05
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -1.9369      0.067    -29.123      0.000      -2.067      -1.807
crossing               0.0214      0.001     25.745      0.000       0.020       0.023
potential              0.3685      0.001    263.374      0.000       0.366       0.371
finishing              0.0104      0.001     11.469      0.000       0.009       0.012
heading_accuracy       0.0683      0.001     84.169      0.000       0.067       0.070
short_passing          0.0482      0.001     35.380      0.000       0.046       0.051
volleys                0.0046      0.001      5.741      0.000       0.003       0.006
dribbling             -0.0123      0.001    -10.639      0.000      -0.015      -0.010
curve                  0.0117      0.001     14.915      0.000       0.010       0.013
free_kick_accuracy     0.0122      0.001     17.417      0.000       0.011       0.014
long_passing           0.0069      0.001      7.440      0.000       0.005       0.009
ball_control           0.1350      0.002     86.083      0.000       0.132       0.138
acceleration           0.0045      0.001      3.328      0.001       0.002       0.007
sprint_speed           0.0087      0.001      6.632      0.000       0.006       0.011
agility               -0.0102      0.001    -10.461      0.000      -0.012      -0.008
reactions              0.2101      0.001    196.151      0.000       0.208       0.212
balance                0.0045      0.001      5.920      0.000       0.003       0.006
shot_power             0.0166      0.001     19.013      0.000       0.015       0.018
jumping                0.0163      0.001     22.929      0.000       0.015       0.018
stamina               -0.0056      0.001     -6.963      0.000      -0.007      -0.004
strength               0.0567      0.001     70.324      0.000       0.055       0.058
long_shots            -0.0120      0.001    -13.252      0.000      -0.014      -0.010
aggression             0.0200      0.001     29.321      0.000       0.019       0.021
interceptions          0.0134      0.001     18.351      0.000       0.012       0.015
positioning           -0.0086      0.001    -11.053      0.000      -0.010      -0.007
vision                -0.0010      0.001     -1.176      0.239      -0.003       0.001
penalties              0.0135      0.001     18.060      0.000       0.012       0.015
marking                0.0315      0.001     28.489      0.000       0.029       0.034
standing_tackle        0.0027      0.001      2.147      0.032       0.000       0.005
sliding_tackle        -0.0278      0.001    -28.364      0.000      -0.030      -0.026
gk_diving              0.1661      0.001    114.616      0.000       0.163       0.169
gk_handling            0.0298      0.002     15.500      0.000       0.026       0.034
gk_kicking            -0.0315      0.0

In [ ]:
# Because of insignificant p-value, let's not consider 'vision' as a feature.

In [47]:
lm = smf.ols(formula = 'overall_rating ~ crossing + potential  + finishing + heading_accuracy + short_passing + \
             volleys + dribbling + curve + free_kick_accuracy + long_passing + ball_control + \
             acceleration + sprint_speed + agility + reactions + balance + shot_power + jumping + \
             stamina + strength + long_shots + aggression + interceptions + positioning + \
             penalties + marking + standing_tackle + sliding_tackle + gk_diving + gk_handling + \
             gk_kicking + gk_positioning + gk_reflexes', data = df).fit()
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         overall_rating   R-squared:                       0.889
Model:                            OLS   Adj. R-squared:                  0.889
Method:                 Least Squares   F-statistic:                 4.451e+04
Date:                Thu, 25 Oct 2018   Prob (F-statistic):               0.00
Time:                        23:47:38   Log-Likelihood:            -4.5073e+05
No. Observations:              183978   AIC:                         9.015e+05
Df Residuals:                  183944   BIC:                         9.019e+05
Df Model:                          33                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -1.9312      0.066    -29.114      0.000      -2.061      -1.801
crossing               0.0215      0.001     25.854      0.000       0.020       0.023
potential              0.3684      0.001    263.515      0.000       0.366       0.371
finishing              0.0104      0.001     11.436      0.000       0.009       0.012
heading_accuracy       0.0684      0.001     84.466      0.000       0.067       0.070
short_passing          0.0480      0.001     35.558      0.000       0.045       0.051
volleys                0.0045      0.001      5.626      0.000       0.003       0.006
dribbling             -0.0122      0.001    -10.601      0.000      -0.015      -0.010
curve                  0.0115      0.001     14.896      0.000       0.010       0.013
free_kick_accuracy     0.0122      0.001     17.405      0.000       0.011       0.014
long_passing           0.0068      0.001      7.346      0.000       0.005       0.009
ball_control           0.1349      0.002     86.138      0.000       0.132       0.138
acceleration           0.0047      0.001      3.479      0.001       0.002       0.007
sprint_speed           0.0087      0.001      6.713      0.000       0.006       0.011
agility               -0.0104      0.001    -10.818      0.000      -0.012      -0.008
reactions              0.2101      0.001    196.158      0.000       0.208       0.212
balance                0.0044      0.001      5.805      0.000       0.003       0.006
shot_power             0.0167      0.001     19.114      0.000       0.015       0.018
jumping                0.0163      0.001     22.913      0.000       0.015       0.018
stamina               -0.0057      0.001     -7.048      0.000      -0.007      -0.004
strength               0.0566      0.001     70.334      0.000       0.055       0.058
long_shots            -0.0120      0.001    -13.263      0.000      -0.014      -0.010
aggression             0.0200      0.001     29.310      0.000       0.019       0.021
interceptions          0.0133      0.001     18.314      0.000       0.012       0.015
positioning           -0.0088      0.001    -11.347      0.000      -0.010      -0.007
penalties              0.0134      0.001     18.025      0.000       0.012       0.015
marking                0.0316      0.001     28.671      0.000       0.029       0.034
standing_tackle        0.0028      0.001      2.237      0.025       0.000       0.005
sliding_tackle        -0.0280      0.001    -28.816      0.000      -0.030      -0.026
gk_diving              0.1662      0.001    114.787      0.000       0.163       0.169
gk_handling            0.0297      0.002     15.484      0.000       0.026       0.034
gk_kicking            -0.0315      0.001    -51.443      0.000      -0.033      -0.030
gk_positioning         0.0516      0.0

In [ ]:
# R - Squared value is not changing much

In [64]:
# Let's go ahead with the above considered features
feature_cols = ['potential', 'crossing',
       'finishing', 'heading_accuracy', 'short_passing', 'volleys',
       'dribbling', 'curve', 'free_kick_accuracy', 'long_passing',
       'ball_control', 'acceleration', 'sprint_speed', 'agility', 'reactions',
       'balance', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots',
       'aggression', 'interceptions', 'positioning', 'vision', 'penalties',
       'marking', 'standing_tackle', 'sliding_tackle', 'gk_diving',
       'gk_handling', 'gk_kicking', 'gk_positioning', 'gk_reflexes']

In [65]:
x = df[feature_cols]

In [66]:
y = df.overall_rating

In [67]:
# Split the data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 0)

In [68]:
# Fit the model
regressor = LinearRegression()
regressor.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [69]:
# Predict the rating
predicted_overall_rating = regressor.predict(x_test)

In [72]:
predicted_overall_rating

array([69.29185   , 67.1444256 , 58.71942326, ..., 68.68452941,
       67.85581696, 71.25035763])

In [70]:
# Model evaluation
msr = mean_squared_error(y_test, predicted_overall_rating)
rmsr = np.sqrt(msr)
print('Mean Squared Error = ', msr)
print('Root Mean Squared Error = ', rmsr)

Mean Squared Error =  7.908474872641185
Root Mean Squared Error =  2.8122010725837483
